<a href="https://colab.research.google.com/github/OliverRevilla/Machine-Learning-Python/blob/main/PCA_y_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
link = "/content/drive/MyDrive/Machine Learning Python/CURSO INFOPUC/Controles/Copia de sensor lb.csv"
dataset = pd.read_csv(link, sep = ";")
dataset.head()

,Activo_id,ciclo,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,max_ciclo,count_down
0,1,1,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,192,191
1,1,2,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,192,190
2,1,3,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,192,189
3,1,4,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,192,188
4,1,5,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,192,187


In [4]:
dataset = dataset.query("Activo_id <= 20")

In [5]:
dataset['vida'] = dataset.ciclo/ dataset.max_ciclo
dataset['target'] = np.where(dataset.vida < 0.8,0,1)
dataset = dataset.drop(columns = ["vida","max_ciclo","count_down"], axis = 1)
dataset.head()

,Activo_id,ciclo,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,target
0,1,1,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,0
1,1,2,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,0
2,1,3,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,0
3,1,4,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,0
4,1,5,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,0


MODELO PCA

In [6]:
# Matriz de características
X = dataset.iloc[:,1:-1].values
Y = dataset.iloc[:,-1].values

In [10]:
# Dvidir en conjuntos
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

In [11]:
# Escalado de varibles
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_test =  pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

In [12]:
explained_variance

array([0.84535663, 0.13666451])

Algoritmo de Bayes

In [15]:
from sklearn.naive_bayes import GaussianNB
modelo_clasificacion = GaussianNB()
modelo_clasificacion.fit(X_train, Y_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [17]:
# Realizando una predicción
y_pred = modelo_clasificacion.predict(X_test)

In [18]:
# Matriz de confusión
from sklearn.metrics import confusion_matrix
matriz_confusion = confusion_matrix(Y_test,y_pred)
matriz_confusion

array([[635,  26],
       [ 20, 153]])

In [19]:
# Validación K-fol cross
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(modelo_clasificacion, X_train, Y_train,cv = 10)
print(accuracies.mean())
print(accuracies.std())

0.9337217456977935
0.01607831149121384
